# Downloads

In [41]:
#Download Keras 2 versions of software
!pip install tensorflow==2.15.0 --quiet #15 13
!pip install tf_keras==2.15.0 --quiet
!pip install tensorflow-datasets==4.8 --quiet #8
!pip install tensorflow-text==2.15.0 --quiet #15
!pip install transformers==4.17 --quiet #4.40.2 #4.37.2
!pip install sentencepiece


^C
ERROR: Operation cancelled by user
ERROR: Could not find a version that satisfies the requirement tensorflow-text==2.15.0 (from versions: 2.17.0rc0, 2.17.0, 2.18.0rc0, 2.18.0)
ERROR: No matching distribution found for tensorflow-text==2.15.0


In [ ]:
!pip3 install torch torchvision torchaudio

In [ ]:
!pip install tensorflow -U --quiet #15 13
!pip install keras -U --quiet
!pip install tensorflow-datasets -U --quiet #8
!pip install tensorflow-text -U --quiet #15
!pip install transformers -U --quiet #4.40.2 #4.37.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.7.0 which is incompatible.
tensorflow-macos 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.4.1 which is incompatible.
tensorflow-macos 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.18.0 which is incompatible.


In [ ]:
!pip install pyarrow==14.0.2 --quiet
!pip install -q datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.1.0 requires pyarrow>=15.0.0, but you have pyarrow 14.0.2 which is incompatible.


In [ ]:
!pip install -q git+https://github.com/keras-team/keras-nlp.git --upgrade

In [ ]:
!pip install -U scikit-learn

In [ ]:
!pip install pydot

# Data Exploration

In [ ]:
from datasets import load_dataset
import pandas as pd

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

dataset = load_dataset("wmt/wmt19", "de-en", split = "train")

In [ ]:
dataset_val = load_dataset("wmt/wmt19", "de-en", split = "validation")

In [ ]:
dataset[0]['translation']

{'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}

# Generating NMT Trtanslations using M2M100

In [ ]:
from transformers import pipeline
m2m100_de_en = pipeline('translation', 'facebook/m2m100_418M', src_lang='de', tgt_lang="en", device="mps", max_length=800)


### Train

In [ ]:
import json

m2m100_master_list = []
for j in range(100):
    subset_i = [dataset[i]['translation']['de'] for i in range(j*1000, (j+1)*1000)]

    subset_i_translated = m2m100_de_en(subset_i)

    # Save the list to a JSON file
    file_name_j = f"m2m100_list_{j}.json"
    with open(file_name_j, "w") as f:
        json.dump(subset_i_translated, f)

    m2m100_master_list.append(subset_i_translated)
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [ ]:
german_text_list = [dataset[j]['translation']['de'] for j in range(100000)]
english_text_list = [dataset[j]['translation']['en'] for j in range(100000)]

In [ ]:
df = pd.read_json('m2m100_full_list.json')
df = pd.DataFrame.from_records(df.values.flatten())
df['german_text'] = german_text_list
df['actual_translation'] = english_text_list
df.to_csv('100000_translation_sample.csv')

In [ ]:
df = pd.read_csv('100000_translation_sample.csv')

In [ ]:
df.iloc[[20,34203,57323,92394],:]

,Unnamed: 0,translation_text,german_text,actual_translation
20,20,"His courageous intervention, unfortunately, co...",Sein mutiges Einschreiten hat ihn bedauerliche...,His courageous actions unfortunately cost him ...
34203,34203,"Furthermore, closer cooperation between the in...",Notwendig ist ferner eine engere Zusammenarbei...,It is also necessary for the different observa...
57323,57323,"Now, the immediate response of the criminal or...","Nun, die unmittelbare Antwort der kriminellen ...",The immediate response of this criminal organi...
92394,92394,I expect much more from the governments that a...,"Ich erwarte von den Regierungen, die sich des ...",I expect much more from governments that are a...


In [ ]:
df.iloc[34203,:].values

array([34203,
       'Furthermore, closer cooperation between the individual observation teams is needed.',
       'Notwendig ist ferner eine engere Zusammenarbeit zwischen den einzelnen Beobachtungsteams.',
       'It is also necessary for the different observation teams to work together more closely.'],
      dtype=object)

In [ ]:
df = df.loc[:, ['translation_text', 'german_text', 'actual_translation']]
df.to_csv('100000_translation_sample.csv', index = False)

### Validation

In [ ]:
validation_german = [dataset_val[i]['translation']['de'] for i in range(1500)]
validation_english = [dataset_val[i]['translation']['en'] for i in range(1500)]

m2m100_validation_translations = m2m100_de_en(validation_german)

NameError: name 'json' is not defined

In [ ]:
# Save the list to a JSON file
import json
file_name = "m2m100_validation_translations.json"
with open(file_name, "w") as f:
    json.dump(m2m100_validation_translations, f)

In [ ]:
val_df = pd.read_json('m2m100_validation_translations.json')
val_df['german_text'] = validation_german
val_df['actual_translation'] = validation_english
val_df.to_csv('1500_validation_translations.csv')

In [ ]:
val_df

,translation_text,german_text,actual_translation
0,Munich 1856: Four maps that change your view o...,"München 1856: Vier Karten, die Ihren Blick auf...",Munich 1856: Four maps that will change your v...
1,This is a place where young people are to meet...,"Eine Irren-Anstalt, wo sich heute Jugendliche ...","A mental asylum, where today young people are ..."
2,"A mining chapel, where it is now buried for th...","Eine Gruftkapelle, wo nun für den S-Bahn-Tunne...","A crypt chapel, where they are now digging tun..."
3,The small gardens rely on the former cause of ...,Kleingärtner bewirtschaften den einstigen Grun...,Allotment holders cultivate the soil of former...
4,The oldest official map of Munich promotes exc...,Die älteste offizielle Karte Münchens fördert ...,The oldest official map of Munich brings capti...
...,...,...,...
1495,They should also provide documents.,Zudem sollten sie Dokumente bereitstellen.,They were also supposed to make documents avai...
1496,"Besides Trump Jr., Trump's adviser and son Jar...",Neben Trump Jr. waren auch Trumps Berater und ...,"In addition to Trump Jr., Trump's adviser and ..."
1497,Legal experts told U.S. television on Thursday...,Rechtsexperten sagten am Donnerstagabend im US...,Legal experts said on US television on Thursda...
1498,"However, it is not necessarily to say that the...","Es müsse aber nicht zwangsläufig heißen, dass ...","However, this does not necessarily mean that t..."


In [43]:
[f"<|im_start|>user\nCorrect the translation: '{question}'<|im_end|>\n<|im_start|>assistant:" for question in val_df['translation_text']]

["<|im_start|>user\nCorrect the translation: 'Munich 1856: Four maps that change your view of the city'<|im_end|>\n<|im_start|>assistant:",
 "<|im_start|>user\nCorrect the translation: 'This is a place where young people are to meet today.'<|im_end|>\n<|im_start|>assistant:",
 "<|im_start|>user\nCorrect the translation: 'A mining chapel, where it is now buried for the S-Bahn tunnel.'<|im_end|>\n<|im_start|>assistant:",
 "<|im_start|>user\nCorrect the translation: 'The small gardens rely on the former cause of farmers.'<|im_end|>\n<|im_start|>assistant:",
 "<|im_start|>user\nCorrect the translation: 'The oldest official map of Munich promotes exciting stories to days.'<|im_end|>\n<|im_start|>assistant:",
 "<|im_start|>user\nCorrect the translation: 'It’s annoying when the maps are not up to date.'<|im_end|>\n<|im_start|>assistant:",
 "<|im_start|>user\nCorrect the translation: 'Everyone knows this, who had to be excited, because the car navy showed a green veil instead of a surrounding 